In [1]:
from mpl_toolkits.mplot3d import Axes3D         # Cargo Axes3D de mpl_toolkits.mplot3d
import matplotlib.pyplot as plt
import numpy as np
import random
import time
import pandas as pd
from matplotlib import animation
import sys 
import math
%matplotlib inline

In [3]:
#Libreria para programar en parallelo
from pyspark import SparkContext
import multiprocessing as mp
from joblib import Parallel, delayed

# Forma 1

In [15]:
#FUNCIÓN DE PARA HACER HISTOGRAMAS 
def Histos_XXX(p,bn,point_max):
    """ 
    Función para construir los histogramas 
    
    p = datos
    bn = tamaño de bins
    point_max = punto máximo en el histograma
    
    """
    
    #Inicializamos los arreglos de los histogramas
    NDDD = np.zeros((bn,bn,bn))
    
    # =========================================================================================== 
    
    n = 0
    for (ii) in zip(p):
        n = n+1
        
        s = ii-p[n:] # Diferencia entre el punto pivote y los demas puntos siguientes 
        r = np.sqrt(s[:,0]**2+s[:,1]**2+s[:,2]**2)
        
        m = 0
        for (kk) in zip(p[n:]):
            
            #DDD ===============================================================================
            r12 = np.ones_like(r[m+1:])*r[m]
            r13 = r[m+1:]
        
            ss = kk-p[n:][m+1:] # Diferencia entre el punto pivote y los demas puntos siguientes 
            r32 = np.sqrt(ss[:,0]**2+ss[:,1]**2+ss[:,2]**2)
        
            # Histogramas para DDD
            dis, dim = np.histogramdd(np.array([r12,r13,r32]).T,bins = [bn,bn,bn], 
                                    range=[[0,point_max],[0,point_max],[0,point_max]])
            NDDD = NDDD + dis
            
            m = m+1
    return NDDD

In [27]:
#FUNCIÓN DE PARA HACER HISTOGRAMAS 
def Histos_XXY(datos,bn,point_max):
    
    p = datos[0]
    p_r = datos[1]
    
    """ 
    Función para construir los histogramas 
    
    p = datos
    p_r = random
    bn = tamaño de bins
    point_max = punto máximo en el histograma
    
    """
    
    #Inicializamos los arreglos de los histogramas
    NDDR = np.zeros((bn,bn,bn))
    
    # =========================================================================================== 
    
    n = 0
    for (ii, jj) in zip(p, p_r):
        n = n+1
        
        s = ii-p[n:] # Diferencia entre el punto pivote y los demas puntos siguientes 
        r = np.sqrt(s[:,0]**2+s[:,1]**2+s[:,2]**2)
        
        m = 0
        for (kk, ll) in zip(p[n:], p_r[n:]):
            
            #DDR ===============================================================================
            r12 = np.ones_like(p_r.T[0])*r[m]
            
            ss = ii-p_r # Diferencia entre el punto pivote y el punto en la muestra random 
            r13 = np.sqrt(ss[:,0]**2+ss[:,1]**2+ss[:,2]**2)
            
            ss = kk-p_r # Diferencia entre el punto 2 de los datos y el punto en la muestra random 
            r32 = np.sqrt(ss[:,0]**2+ss[:,1]**2+ss[:,2]**2)
            
            # Histogramas para DDR
            dis, dim = np.histogramdd(np.array([r12,r13,r32]).T,bins = [bn,bn,bn], 
                                    range=[[0,point_max],[0,point_max],[0,point_max]])
            NDDR = NDDR + dis
            
            m = m+1
    
    return  NDDR

In [19]:
data = np.loadtxt('/home/echeveste/mis_trabajos/correlation_f/data/data_500.dat')[:,:3]
rand = np.loadtxt('/home/echeveste/mis_trabajos/correlation_f/data/rand0_500.dat')[:,:3]

In [20]:
sc.stop()

In [21]:
nuc = 2
sc = SparkContext(master='local[{0}]'.format(int(nuc)))
print(sc.master)

local[2]


In [22]:
data_rand = sc.parallelize(np.split(np.concatenate((data, rand)),2))

In [17]:
#Paralelizado
start = time.perf_counter()

DDD = Histos_XXX(data,30,180)
RRR = Histos_XXX(rand,30,180)

finish = time.perf_counter()

print(f'Finializó en {round(finish-start,2)} segundos\n')

Finializó en 69.61 segundos



In [18]:
#Paralelizado
start_pll = time.perf_counter()

XXX = data_rand.map(lambda x: Histos_XXX(x,30,180)).collect()

finish_pll = time.perf_counter()
sc.stop()
print(f'Finializó en {round(finish_pll-start_pll,2)} segundos\n')

Finializó en 38.82 segundos



In [30]:
DDD = XXX[0]
RRR = XXX[1]

In [23]:
print(f'Paralelizado fue {round((finish-start)/(finish_pll-start_pll),2)} veces mas rapido\n')

Paralelizado fue 1.79 veces mas rapido



In [29]:
#Paralelizado
start_pll = time.perf_counter()

XXY = Parallel(n_jobs=nuc)(delayed(Histos_XXY)(ii,30,180) for ii in [[data,rand],[rand,data]])

finish_pll = time.perf_counter()
print(f'Finializó en {round(finish_pll-start_pll,2)} segundos\n')

Finializó en 42.48 segundos



In [32]:
DDR = XXY[0]
DRR = XXY[1]

# Forma 2

De esta forma tenemos la libertad de agregar mas nucleos si los tuvieramos. Por ahora solo usaremos 2 de los cuatro

In [86]:
def puntos_23(ii,p,NDDD,bn,point_max):
    
    s = ii-p 
    r = np.sqrt(s[:,0]**2+s[:,1]**2+s[:,2]**2)
        
    m = 0
    for (kk) in zip(p):
            
        r12 = np.ones_like(r[m+1:])*r[m]
        r13 = r[m+1:]
    
        ss = kk-p[m+1:]
        r32 = np.sqrt(ss[:,0]**2+ss[:,1]**2+ss[:,2]**2)
        
        dis, dim = np.histogramdd(np.array([r12,r13,r32]).T,bins = [bn,bn,bn], 
                                  range=[[0,point_max],[0,point_max],[0,point_max]])
        NDDD = NDDD + dis
        
        m = m+1
    
    return NDDD

In [87]:
#FUNCIÓN DE PARA HACER HISTOGRAMAS 
def Histos_XXX(p,bn,point_max,nuc):
    """ 
    Función para construir los histogramas 
    
    p = datos
    bn = tamaño de bins
    point_max = punto máximo en el histograma
    nuc = cantidad de núcleos implementados 
    
    """
    
    #Inicializamos el arreglo del histograma
    NDDD = np.zeros((bn,bn,bn))
    
    return np.sum(np.array(Parallel(n_jobs=nuc)(delayed(puntos_23)(p[i],p[i+1:],NDDD,30,180) for i in range(len(p)))),axis=1)
    
    

In [88]:
#Paralelizado
start_pll = time.perf_counter()

DDD_2 = Histos_XXX(data,30,180,2)
RRR_2 = Histos_XXX(rand,30,180,2)

finish_pll = time.perf_counter()
print(f'Finializó en {round(finish_pll-start_pll,2)} segundos\n')

Finializó en 38.22 segundos



Como vemos, el tiempo en el que se realizan los histograms es practicamente el mismo que en la primera forma de paralelizar. Por lo tanto, esta forma es mejor por la capasidad de agregar más núcleos para mayor eficiencia.